In [1]:
%load_ext autoreload
%autoreload 2

In [44]:
import numpy as np
import torch
import time
import sys
import resource
from torch.utils.data import DataLoader

from data_classes import *
from read_input import *
from read_trainset import *
from network import *
from prepare_batches import *
from traininit import *
from data_set import *
from data_loader import *
from optimization_step import *
from output_nn import *
from py_aeio import *

In [45]:
device = "cpu"

In [46]:
tin_file = "/nethome/farri002/mlp-pt-au-o-h/ml-construction/PdO/desc-20-8/NN-20-20/train.in"

In [47]:
tin_file = "/nethome/farri002/bin/forks/aenet-PyTorch/example/02-train/train.in"
tin = read_train_in(tin_file)
torch.manual_seed(3)
np.random.seed(tin.numpy_seed)
if tin.verbose: io_input_reading(tin)
tin.train_file = '/nethome/farri002/bin/forks/aenet-PyTorch/example/01-generate/TiO.train.ascii'
tin.train_forces_file = '/nethome/farri002/bin/forks/aenet-PyTorch/example/01-generate/TiO.train.forces'

----------------------------------------------------------------------
                       Reading input information                      
----------------------------------------------------------------------

Reading input parameters.
These are the parameters selected for training:
        - TRAININGSET: TiO.train.ascii
        - TESTPERCENT: 0.1
        - ITERATIONS:  500
        - ITERWRITE:   1
        - BATCHSIZE:   128
        - MEMORY_MODE: cpu

        - FORCES:      True
        - alpha:       0.1



In [48]:
list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)

In [49]:
N_removed = len(list_removed)
N_struc_E = len(list_structures_energy)
N_struc_F = len(list_structures_forces)

In [50]:
def select_batches(tin, trainset_params, device, list_structures_energy, list_structures_forces,
				   max_nnb, N_batch_train, N_batch_test, N_batch_valid, train_sampler_E, test_sampler_E, valid_sampler_E):
        """
        Select which structures belong to each batch for training.
        Returns: four objects of the class data_set_loader.PrepDataloader(), for train/test and energy/forces
        """
        dataset_energy = StructureDataset(list_structures_energy, tin.sys_species, tin.networks_param["input_size"], max_nnb)
        
        dataset_energy_size = len(dataset_energy)

        # Normalize
        E_scaling, E_shift = tin.trainset_params.E_scaling, tin.trainset_params.E_shift
        sfval_avg, sfval_cov = tin.setup_params.sfval_avg, tin.setup_params.sfval_cov
        dataset_energy.normalize_E(trainset_params.E_scaling, trainset_params.E_shift)
        stp_shift, stp_scale = dataset_energy.normalize_stp(sfval_avg, sfval_cov)

        # Split in train/test
        #train_sampler_E, valid_sampler_E = split_database(dataset_energy_size, tin.test_split)

        train_energy_data = PrepDataloader(dataset=dataset_energy, train_forces=False, N_batch=N_batch_train,
                                        sampler=train_sampler_E, memory_mode=tin.memory_mode, device=device, dataname="train_energy")
        test_energy_data = PrepDataloader(dataset=dataset_energy, train_forces=False, N_batch=N_batch_train,
                                        sampler=test_sampler_E, memory_mode=tin.memory_mode, device=device, dataname="test_energy")

        if valid_sampler_E:
            valid_energy_data = PrepDataloader(dataset=dataset_energy, train_forces=False, N_batch=N_batch_valid,
                                        sampler=valid_sampler_E, memory_mode=tin.memory_mode, device=device, dataname="valid_energy")
            return train_energy_data, test_energy_data, valid_energy_data
        
        return train_energy_data, test_energy_data

In [51]:
def get_sets_from_indices(training_indices, test_indices, valid_indices, list_structures_energy):
    N_batch_train, N_batch_test, N_batch_valid = 1, 1, 1 #select_batch_size(tin, list_structures_energy, list_structures_forces)

    # Join datasets with forces and only energies in a single torch dataset AND prepare batches
    if valid_indices:
        train_energy_data, test_energy_data, valid_energy_data = select_batches(tin, tin.trainset_params, device, list_structures_energy, None,
                    max_nnb, N_batch_train,N_batch_test, N_batch_valid, training_indices, test_indices, valid_indices)
    else:
        train_energy_data, test_energy_data = select_batches(tin, tin.trainset_params, device, list_structures_energy, None,
                    max_nnb, N_batch_train,N_batch_test, N_batch_valid, training_indices, test_indices, valid_indices)
        
    grouped_train_data = GroupedDataset(train_energy_data, None,
									 memory_mode=tin.memory_mode, device=device, dataname="train")
    grouped_test_data = GroupedDataset(test_energy_data, None,
									 memory_mode=tin.memory_mode, device=device, dataname="test")
    
    grouped_train_loader = DataLoader(grouped_train_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)
    grouped_test_loader = DataLoader(grouped_test_data, batch_size=1, shuffle=False,
                                  collate_fn=custom_collate, num_workers=0)

    if valid_indices:
        grouped_valid_data = GroupedDataset(valid_energy_data, None,
									    memory_mode=tin.memory_mode, device=device, dataname="valid")
        grouped_valid_loader = DataLoader(grouped_valid_data, batch_size=1, shuffle=False,
                                    collate_fn=custom_collate, num_workers=0)
        
        return grouped_train_loader, grouped_test_loader, grouped_valid_loader
    
    return grouped_train_loader, grouped_test_loader

In [52]:
np.random.seed(42)

dataset_size = len(list_structures_energy)
indices = list(range(dataset_size))
np.random.shuffle(indices)

In [53]:
training_indices, test_indices, valid_indices = indices[:650], indices[650:670], indices[670:]

In [54]:
import copy

grouped_train_loader, grouped_test_loader, grouped_valid_loader = get_sets_from_indices(training_indices, test_indices, valid_indices, copy.deepcopy(list_structures_energy))

In [55]:
def get_train_test(grouped_train_loader):
    for data_batch in grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce, grp_N_atoms = data_batch[0][10], data_batch[0][11], data_batch[0][12],data_batch[0][14]

    grp_descrp[0] = grp_descrp[0].float()
    grp_descrp[1] = grp_descrp[1].float()

    logic_reduce[0] = logic_reduce[0].float()
    logic_reduce[1] = logic_reduce[1].float()

    return grp_descrp, grp_energy, logic_reduce, grp_N_atoms

In [56]:
def summary(samples):
    site_stats = {}
    for k, v  in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0).numpy(),
            "std": torch.std(v, 0).numpy(),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0].numpy(),
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0].numpy(),
        }
    return site_stats

In [65]:
from bnn import BayesianNeuralNetwork
from pyro.infer import SVI, Trace_ELBO, TraceMeanField_ELBO
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import Predictive
import pyro

net = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)

bnn = BayesianNeuralNetwork(net)

In [ ]:
v_grp_descrp, v_grp_energy, v_logic_reduce, v_grp_N_atoms = get_train_test(grouped_valid_loader) 
t_grp_descrp, t_grp_energy, t_logic_reduce, t_grp_N_atoms = get_train_test(grouped_test_loader) 

In [102]:
for i in range(15,100):
    loss = bnn.train(grouped_train_loader, 10000)

    predictions = bnn.predict(v_grp_descrp, v_logic_reduce, num_samples=800)
    std = torch.std(predictions['obs'], 0).numpy()
    l2, std_l2 = bnn.get_loss_RMSE(v_grp_descrp, v_grp_energy, v_logic_reduce, v_grp_N_atoms, 10000)

    print(' ')
    print('Epochs: {}'.format(i*10000))
    print('loss: {}'.format(loss))
    print('Std Valid Set: {}'.format(np.mean(std)))
    print('RMSD: {} +- {}'.format(l2, std_l2))
    

 
Epochs: 150000
loss: 6.442596814291762
Std Valid Set: 2.141653437818345
RMSD: 2.5522304161989795 +- 1.2398254975827705
 
Epochs: 160000
loss: 2.934505159575083
Std Valid Set: 2.1240868126145895
RMSD: 2.53533429111407 +- 1.5793293239261244
 
Epochs: 170000
loss: 2.526634696552715
Std Valid Set: 2.056527231734015
RMSD: 2.4754785026919484 +- 1.5783095101654219
 
Epochs: 180000
loss: 2.4731265591537146
Std Valid Set: 2.355816084897385
RMSD: 2.4531397124076184 +- 1.7074034547341037
 
Epochs: 190000
loss: 2.7274045096437307
Std Valid Set: 2.0731383056385204
RMSD: 2.4096378053338814 +- 1.363494939815068
 
Epochs: 200000
loss: 2.4899567276352315
Std Valid Set: 2.14549511273274
RMSD: 2.328349932175319 +- 1.4822601329544678
 
Epochs: 210000
loss: 2.497458184528762
Std Valid Set: 1.9757761262808842
RMSD: 2.2992955819414247 +- 1.8279626179560577
 
Epochs: 220000
loss: 3.4269197478007074
Std Valid Set: 2.079203725289267
RMSD: 2.237810065600217 +- 2.5988219853113432
 
Epochs: 230000
loss: 2.486922

In [132]:
for n in range(1, 21):
    t_predictions = bnn_trained.predict(t_grp_descrp, t_logic_reduce, num_samples=1000*n)

    print('\nNumber of Samples: {}'.format(n*1000))
    print('Mean value obs: {}'.format(torch.mean(t_predictions['obs'][:,1], 0).numpy()))
    print('STD obs: {}'.format(torch.std(t_predictions['obs'][:,1], 0).numpy()))

    print('Mean value obs: {}'.format(torch.mean(t_predictions['_RETURN'][:,1], 0).numpy()))
    print('STD obs: {}'.format(torch.std(t_predictions['_RETURN'][:,1], 0).numpy()))


Number of Samples: 1000
Mean value obs: -13.131354121911505
STD obs: 0.654095921256454
Mean value obs: -13.110956069946289
STD obs: 0.4167194912896756

Number of Samples: 2000
Mean value obs: -13.124368621049998
STD obs: 0.671856899313604
Mean value obs: -13.116442487716675
STD obs: 0.4257772129384873

Number of Samples: 3000
Mean value obs: -13.118869058179847
STD obs: 0.8574290701068339
Mean value obs: -13.116594337463379
STD obs: 0.6817847447063423

Number of Samples: 4000
Mean value obs: -13.125151220920529
STD obs: 0.7192836934499853
Mean value obs: -13.114072032749652
STD obs: 0.5159731383263086

Number of Samples: 5000
Mean value obs: -13.100050400678601
STD obs: 0.708620411133381
Mean value obs: -13.108314805603028
STD obs: 0.48953026429733726

Number of Samples: 6000
Mean value obs: -13.110401888990156
STD obs: 0.7876330574037411
Mean value obs: -13.11762721979618
STD obs: 0.6085350247619573

Number of Samples: 7000
Mean value obs: -13.122082464748484
STD obs: 0.8396672206943

In [149]:
net = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)

bnn = BayesianNeuralNetwork(net)

bnn.train(grouped_train_loader, 10000)

4.26069700348863

In [150]:
v_predictions = bnn.predict(v_grp_descrp, v_logic_reduce, num_samples=800)
t_predictions = bnn.predict(t_grp_descrp, t_logic_reduce, num_samples=800)

t_std = torch.std(t_predictions['obs'], 0).numpy()
v_std = torch.std(v_predictions['obs'], 0).numpy()

print(t_std)

[3.16938014 4.00191566 3.62253959 3.38525584 3.73111089 5.38229097
 3.99725794 3.88566738 3.84382997 3.23328602 3.96624393 3.73362059
 3.96680378 3.30055488 3.45105927 3.91641656 4.82355363 3.63048142
 3.68462159 3.70996793]


In [151]:
for i, std_i in zip(test_indices, t_std): 
    new_indinces = training_indices + [i]
    new_grouped_train_loader, _, _ = get_sets_from_indices(new_indinces, test_indices, valid_indices, list_structures_energy)
    
    bnn_trained = copy.deepcopy(bnn)

    bnn_trained.train(new_grouped_train_loader, 10000)

    v_predictions = bnn_trained.predict(v_grp_descrp, v_logic_reduce, num_samples=800)
    v_std = torch.std(v_predictions['obs'], 0).numpy()

    print(std_i, np.mean(v_std))

3.1693801428269697 9.773642700932248
4.001915662242675 9.779298349244522
3.622539585439741 9.80644698051442
3.3852558423642294 9.798003385848006
3.7311108892894254 9.773315119444723
5.382290968723228 9.814592696415906
3.997257939634929 9.800738971808016
3.8856673825914996 9.852944005258575
3.843829974772948 9.807626130796052
3.2332860241550763 9.807246663172952
3.9662439312957245 9.747403295044112
3.7336205897704318 9.804015795756394
3.9668037833987944 9.817125330597326
3.300554878018853 9.833374874343239
3.451059269631064 9.824919554464017
3.9164165615030466 9.748966140591872
4.823553628776083 9.803109060760892
3.630481415706456 9.806264675505801
3.6846215864646124 9.769581502733605


In [ ]:
for x in sorted_std_test[-5:]
guided_choice = [test_indices[x] for x in sorted_std_test[-5:]] + training_indices


In [62]:
v_grp_descrp, v_grp_energy, v_logic_reduce, v_grp_N_atoms = get_train_test(grouped_valid_loader) 
t_grp_descrp, t_grp_energy, t_logic_reduce, t_grp_N_atoms = get_train_test(grouped_test_loader) 

In [16]:
pyro.clear_param_store()
num_steps = 30000
initial_lr = 0.001
gamma = 0.1  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
optim = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
svi = SVI(bnn.model, guide, optim, loss=TraceMeanField_ELBO())

for j in range(num_steps):
    loss = 0
    for data_batch in grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
        grp_descrp[0] = grp_descrp[0].float()
        grp_descrp[1] = grp_descrp[1].float()

        logic_reduce[0] = logic_reduce[0].float()
        logic_reduce[1] = logic_reduce[1].float()
        loss += svi.step(grp_descrp, logic_reduce, grp_energy) 
    if j % 100 == 0:
        print("[EPOCH LOSS %04d] loss: %.4f" % (j + 1, loss / len(logic_reduce[0])))

[EPOCH LOSS 0001] loss: 17.0463
[EPOCH LOSS 0101] loss: 7.7749


[EPOCH LOSS 0201] loss: 4.8907
[EPOCH LOSS 0301] loss: 4.6197
[EPOCH LOSS 0401] loss: 4.5332
[EPOCH LOSS 0501] loss: 4.4094
[EPOCH LOSS 0601] loss: 4.3450
[EPOCH LOSS 0701] loss: 4.2393
[EPOCH LOSS 0801] loss: 4.4226
[EPOCH LOSS 0901] loss: 4.4196
[EPOCH LOSS 1001] loss: 4.3615
[EPOCH LOSS 1101] loss: 4.0253
[EPOCH LOSS 1201] loss: 3.9867
[EPOCH LOSS 1301] loss: 4.0377
[EPOCH LOSS 1401] loss: 3.9345
[EPOCH LOSS 1501] loss: 3.7175
[EPOCH LOSS 1601] loss: 3.8345
[EPOCH LOSS 1701] loss: 3.8607
[EPOCH LOSS 1801] loss: 3.6053
[EPOCH LOSS 1901] loss: 3.7277
[EPOCH LOSS 2001] loss: 3.4045
[EPOCH LOSS 2101] loss: 3.8286
[EPOCH LOSS 2201] loss: 3.6320
[EPOCH LOSS 2301] loss: 3.4881
[EPOCH LOSS 2401] loss: 5.6838
[EPOCH LOSS 2501] loss: 3.3327
[EPOCH LOSS 2601] loss: 3.4428
[EPOCH LOSS 2701] loss: 3.4388
[EPOCH LOSS 2801] loss: 3.4777
[EPOCH LOSS 2901] loss: 3.8206
[EPOCH LOSS 3001] loss: 4.6018
[EPOCH LOSS 3101] loss: 3.3301
[EPOCH LOSS 3201] loss: 3.3076
[EPOCH LOSS 3301] loss: 3.1087
[EPOCH L

In [17]:
del grouped_train_loader

In [18]:
predictive = Predictive(bnn.model, guide=guide, num_samples=8000,
                        return_sites=("obs", "_RETURN"))

v_grp_descrp, v_grp_energy, v_logic_reduce, v_grp_N_atoms = get_train_test(grouped_valid_loader) 
t_grp_descrp, t_grp_energy, t_logic_reduce, t_grp_N_atoms = get_train_test(grouped_test_loader)    

v_samples = predictive(v_grp_descrp, v_logic_reduce)
t_samples = predictive(t_grp_descrp, t_logic_reduce)

In [19]:
v_std = torch.std(v_samples['obs'], 0).numpy()
t_std = torch.std(t_samples['obs'], 0).numpy()

print('Standard Deviation Test Set {}'.format(np.mean(t_std)))
print('Standard Deviation Validation Set {}'.format(np.mean(v_std)))

Standard Deviation Test Set 2.3848565685704215
Standard Deviation Validation Set 2.6130075268125155


In [28]:
sorted_std_test = np.argsort(torch.std(t_samples['obs'], 0))

In [29]:
guided_choice = [test_indices[x] for x in sorted_std_test[-5:]] + training_indices
shuffle_test = np.random.choice(len(test_indices), len(test_indices), replace=False)
random_choice = [test_indices[x] for x in shuffle_test[-5:]] + training_indices

In [30]:
print(torch.std(t_samples['obs'][:,sorted_std_test[-5:]], dim=0))
print(torch.std(t_samples['obs'][:,shuffle_test[-5:]], dim=0))

tensor([2.4415, 2.5250, 2.5796, 2.7677, 3.6075], dtype=torch.float64)
tensor([2.7677, 2.3808, 2.1901, 2.2747, 2.1727], dtype=torch.float64)


In [31]:
#GUIDED CHOICE

net = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)

guided_model = copy.deepcopy(bnn)
guided_guide = copy.deepcopy(guide)

list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)
G_grouped_train_loader, G_grouped_test_loader, G_grouped_valid_loader = get_sets_from_indices(guided_choice, test_indices, valid_indices, list_structures_energy)

In [32]:
pyro.clear_param_store()
num_steps = 30000
initial_lr = 0.001
gamma = 0.1  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
optim = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
svi = SVI(guided_model.model, guided_guide, optim, loss=TraceMeanField_ELBO())

for j in range(num_steps):
    loss = 0
    for data_batch in G_grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
        grp_descrp[0] = grp_descrp[0].float()
        grp_descrp[1] = grp_descrp[1].float()

        logic_reduce[0] = logic_reduce[0].float()
        logic_reduce[1] = logic_reduce[1].float()
        loss += svi.step(grp_descrp, logic_reduce, grp_energy) 
    if j % 100 == 0:
        print("[EPOCH LOSS %04d] loss: %.4f" % (j + 1, loss / len(logic_reduce[0])))

[EPOCH LOSS 0001] loss: 2.7224
[EPOCH LOSS 0101] loss: 3.0565
[EPOCH LOSS 0201] loss: 2.6703
[EPOCH LOSS 0301] loss: 2.7540
[EPOCH LOSS 0401] loss: 2.6830
[EPOCH LOSS 0501] loss: 2.5228
[EPOCH LOSS 0601] loss: 2.6069
[EPOCH LOSS 0701] loss: 2.4956
[EPOCH LOSS 0801] loss: 2.4480
[EPOCH LOSS 0901] loss: 2.5127
[EPOCH LOSS 1001] loss: 2.4854
[EPOCH LOSS 1101] loss: 2.6648
[EPOCH LOSS 1201] loss: 2.6071
[EPOCH LOSS 1301] loss: 2.4373
[EPOCH LOSS 1401] loss: 2.5080
[EPOCH LOSS 1501] loss: 2.6573
[EPOCH LOSS 1601] loss: 2.5926
[EPOCH LOSS 1701] loss: 2.6522
[EPOCH LOSS 1801] loss: 2.5841
[EPOCH LOSS 1901] loss: 2.5841
[EPOCH LOSS 2001] loss: 2.8068
[EPOCH LOSS 2101] loss: 2.9259
[EPOCH LOSS 2201] loss: 2.5486
[EPOCH LOSS 2301] loss: 2.9848
[EPOCH LOSS 2401] loss: 2.7530
[EPOCH LOSS 2501] loss: 2.6728
[EPOCH LOSS 2601] loss: 2.3884
[EPOCH LOSS 2701] loss: 2.7636
[EPOCH LOSS 2801] loss: 2.7813
[EPOCH LOSS 2901] loss: 2.6957
[EPOCH LOSS 3001] loss: 2.7482
[EPOCH LOSS 3101] loss: 2.5000
[EPOCH L

In [33]:
predictive = Predictive(guided_model.model, guide=guided_guide, num_samples=10000,
                        return_sites=("obs", "_RETURN"))

G_v_samples = predictive(v_grp_descrp, v_logic_reduce)
G_v_std = torch.std(G_v_samples['obs'], 0).numpy()

print('STD Validation Set before guided choice: {}'.format(np.mean(v_std)))
print('STD Validation Set after guided choice: {}'.format(np.mean(G_v_std)))

STD Validation Set before guided choice: 2.6130075268125155
STD Validation Set after guided choice: 1.9240081625845111


In [34]:
# Uncertanty guided active learning and random active learning model split
net = NetAtom(tin.networks_param["input_size"], tin.networks_param["hidden_size"],
			    tin.sys_species, tin.networks_param["activations"], tin.alpha, device)

random_model = copy.deepcopy(bnn)
random_guide = copy.deepcopy(guide)

In [35]:
#RANDOM CHOICE
list_structures_energy, list_structures_forces, list_removed, max_nnb, tin = read_list_structures(tin)
R_grouped_train_loader,R_grouped_test_loader, R_grouped_valid_loader = get_sets_from_indices(random_choice, test_indices, valid_indices, list_structures_energy)

In [36]:
pyro.clear_param_store()
num_steps = 30000
initial_lr = 0.001
gamma = 0.1  # final learning rate will be gamma * initial_lr
lrd = gamma ** (1 / num_steps)
optim = pyro.optim.ClippedAdam({'lr': initial_lr, 'lrd': lrd})
svi = SVI(random_model.model, random_guide, optim, loss=Trace_ELBO())

for j in range(num_steps):
    loss = 0
    for data_batch in R_grouped_train_loader:
        grp_descrp, grp_energy, logic_reduce = data_batch[0][10], data_batch[0][11], data_batch[0][12]
        grp_descrp[0] = grp_descrp[0].float()
        grp_descrp[1] = grp_descrp[1].float()

        logic_reduce[0] = logic_reduce[0].float()
        logic_reduce[1] = logic_reduce[1].float()
        loss += svi.step(grp_descrp, logic_reduce, grp_energy) 
    if j % 100 == 0:
        print("[EPOCH LOSS %04d] loss: %.4f" % (j + 1, loss / len(logic_reduce[0])))

[EPOCH LOSS 0001] loss: 2.6842
[EPOCH LOSS 0101] loss: 2.7201
[EPOCH LOSS 0201] loss: 2.7729
[EPOCH LOSS 0301] loss: 2.8243
[EPOCH LOSS 0401] loss: 2.4824
[EPOCH LOSS 0501] loss: 2.6350
[EPOCH LOSS 0601] loss: 2.7592
[EPOCH LOSS 0701] loss: 2.6385
[EPOCH LOSS 0801] loss: 2.6289
[EPOCH LOSS 0901] loss: 3.0178
[EPOCH LOSS 1001] loss: 2.6068
[EPOCH LOSS 1101] loss: 2.6099
[EPOCH LOSS 1201] loss: 2.7412
[EPOCH LOSS 1301] loss: 2.6447
[EPOCH LOSS 1401] loss: 2.5071
[EPOCH LOSS 1501] loss: 2.5039
[EPOCH LOSS 1601] loss: 2.5402
[EPOCH LOSS 1701] loss: 2.8540
[EPOCH LOSS 1801] loss: 2.4633
[EPOCH LOSS 1901] loss: 3.0963
[EPOCH LOSS 2001] loss: 2.5887
[EPOCH LOSS 2101] loss: 2.5670
[EPOCH LOSS 2201] loss: 2.5627
[EPOCH LOSS 2301] loss: 2.6470
[EPOCH LOSS 2401] loss: 2.6647
[EPOCH LOSS 2501] loss: 2.4209
[EPOCH LOSS 2601] loss: 2.6114
[EPOCH LOSS 2701] loss: 2.4511
[EPOCH LOSS 2801] loss: 2.5648
[EPOCH LOSS 2901] loss: 2.4593
[EPOCH LOSS 3001] loss: 2.6376
[EPOCH LOSS 3101] loss: 2.4395
[EPOCH L

In [37]:
del R_grouped_train_loader

In [38]:
predictive = Predictive(random_model.model, guide=random_guide, num_samples=10000,
                        return_sites=("obs", "_RETURN"))

R_v_samples = predictive(v_grp_descrp, v_logic_reduce)
R_v_std = torch.std(R_v_samples['_RETURN'], 0).numpy()


print('STD Validation Set before random choice: {}'.format(np.mean(v_std)))
print('STD Validation Set after choice: {}'.format(np.mean(R_v_std)))

STD Validation Set before random choice: 2.6130075268125155
STD Validation Set after choice: 1.1125461442639568


In [39]:
predictive = Predictive(bnn.model, guide=guide, num_samples=10000,
                        return_sites=("obs", "_RETURN"))
t_samples = predictive(t_grp_descrp, t_logic_reduce)
print(torch.std(t_samples['obs'][:,sorted_std_test[-5:]], dim=0))
print(torch.std(t_samples['obs'][:,shuffle_test[-5:]], dim=0))

tensor([2.6072, 2.8733, 2.3040, 3.5480, 4.0700], dtype=torch.float64)
tensor([3.5480, 2.1885, 2.5797, 2.4523, 2.5608], dtype=torch.float64)


In [42]:
predictive = Predictive(random_model.model, guide=random_guide, num_samples=10000,
                        return_sites=("obs", "_RETURN"))
t_samples = predictive(t_grp_descrp, t_logic_reduce)
print(torch.std(t_samples['obs'][:,sorted_std_test[-5:]], dim=0))
print(torch.std(t_samples['obs'][:,shuffle_test[-5:]], dim=0))

tensor([1.8253, 1.6166, 1.5502, 2.1510, 2.9217], dtype=torch.float64)
tensor([2.1510, 1.6516, 1.5571, 1.6355, 1.5723], dtype=torch.float64)


In [43]:
predictive = Predictive(guided_model.model, guide=guided_guide, num_samples=10000,
                        return_sites=("obs", "_RETURN"))
t_samples = predictive(t_grp_descrp, t_logic_reduce)
print(torch.std(t_samples['obs'][:,sorted_std_test[-5:]], dim=0))
print(torch.std(t_samples['obs'][:,shuffle_test[-5:]], dim=0))

tensor([1.8165, 1.6238, 1.5754, 2.1741, 2.9024], dtype=torch.float64)
tensor([2.1741, 1.6356, 1.5649, 1.6779, 1.5773], dtype=torch.float64)
